# For Everyone

IMPORTS

In [0]:
import argparse
import torch
import torchvision
from torch import optim
from torchvision import transforms
import os
import os.path as osp
import random
import numpy as np
from pathlib import Path
from torch.utils.data import dataset
import PIL
from PIL import Image

# Fix seed to make experiments reproducibles
seed = 1

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(seed)
random.seed(seed)

ACCESS TO THE DRIVE

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir if necessary

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


FOLDER WHERE THE WEIGHTS ARE STORED

In [0]:
# directory where we store the model weights
weights_dir = "gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa/Presentacio/Demo/Weights_here" #change dir if necessary
if not osp.exists(weights_dir):
    os.mkdir(weights_dir)

# Demo

DEFINE ARGUMENTS

In [0]:
class Args:

    frontal_images_directories = "gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa/dataset-cfp/Protocol/image_list_F.txt" #change dir if necessary
    profile_images_directories = "gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa/dataset-cfp/Protocol/image_list_P.txt" #change dir if necessary
    split_main_directory = "gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa/dataset-cfp/Protocol/Split" #change dir if necessary
    split_traindata = ["01", "02", "03", "04", "05", "06"]
    split_valdata = ["07", "08"]
    split_testdata = ["09", "10"]
    dataset_root = "gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa" #change dir if necessary
    dataset= "CFPDataset"
    lr = float(1e-4)
    weight_decay = float(0.0005)
    momentum = float(0.9)
    betas = (0.9, 0.999)
    batch_size = int(14)
    workers = int(8)
    start_epoch = int(0)
    epochs = int(40)
    pretrained = True
    data_aug = False
    resume = "checkpoint_e63_40e_lr1e_3_SGD"


DEFINE DATASET CLASS

In [0]:

class CFPDataset(dataset.Dataset):
    def __init__(self, path, args, img_transforms=None, dataset_root="",
                 split="train", input_size=(224, 224)):
        super().__init__()

        self.data = []
        self.split = split

        self.load(path, args)

        print("Dataset loaded")
        print("{0} samples in the {1} dataset".format(len(self.data),
                                                      self.split))
        self.transforms = img_transforms
        self.dataset_root = dataset_root
        self.input_size = input_size

    def load(self, path, args):

        # read directories for frontal images
        lines = open(args.frontal_images_directories).readlines()
        idx = 0
        directories_frontal_images = []
        while idx < len(lines):
            x = lines[idx].strip().split()
            directories_frontal_images.append(x)
            idx += 1
        
        # read directories for profile images
        lines = open(args.profile_images_directories).readlines()
        idx = 0
        directories_profile_images = []
        while idx < len(lines):
            x = lines[idx].strip().split()
            directories_profile_images.append(x)
            idx += 1
        
        # read same and different pairs of images and save at dictionary
        self.data = []
        for i in path:
            ff_diff_file = osp.join(args.split_main_directory, 'FF', i,
                                    'diff.txt')
            lines = open(ff_diff_file).readlines()
            idx = 0
            while idx < 3: #int(len(lines)/1): changed for inference demo
                img_pair = lines[idx].strip().split(',')
                img1_dir = directories_frontal_images[int(img_pair[0])-1][1]
                img2_dir = directories_frontal_images[int(img_pair[1])-1][1]
                pair_tag = -1
                d = {
                    "img1_path": img1_dir,
                    "img2_path": img2_dir,
                    "pair_tag": pair_tag
                }
                self.data.append(d)
                idx += 1

            ff_same_file = osp.join(args.split_main_directory, 'FF', i,
                                    'same.txt')
            lines = open(ff_same_file).readlines()
            idx = 0
            while idx < 3: #int(len(lines)/1): changed for inference demo
                img_pair = lines[idx].strip().split(',')
                img1_dir = directories_frontal_images[int(img_pair[0])-1][1]
                img2_dir = directories_frontal_images[int(img_pair[1])-1][1]
                pair_tag = 1
                d = {
                    "img1_path": img1_dir,
                    "img2_path": img2_dir,
                    "pair_tag": pair_tag
                }
                self.data.append(d)
                idx += 1

            fp_diff_file = osp.join(args.split_main_directory, 'FP', i,
                                    'diff.txt')
            lines = open(fp_diff_file).readlines()
            idx = 0
            while idx < 3: #int(len(lines)/1): changed for inference demo
                img_pair = lines[idx].strip().split(',')
                img1_dir = directories_frontal_images[int(img_pair[0])-1][1]
                img2_dir = directories_profile_images[int(img_pair[1])-1][1]
                pair_tag = -1
                d = {
                    "img1_path": img1_dir,
                    "img2_path": img2_dir,
                    "pair_tag": pair_tag
                }
                self.data.append(d)
                idx += 1

            fp_same_file = osp.join(args.split_main_directory, 'FP', i,
                                    'same.txt')
            lines = open(fp_same_file).readlines()
            idx = 0
            while idx < 3: #int(len(lines)/1): changed for inference demo
                img_pair = lines[idx].strip().split(',')
                img1_dir = directories_frontal_images[int(img_pair[0])-1][1]
                img2_dir = directories_profile_images[int(img_pair[1])-1][1]
                pair_tag = 1
                d = {
                    "img1_path": img1_dir,
                    "img2_path": img2_dir,
                    "pair_tag": pair_tag
                }
                self.data.append(d)
                idx += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        d = self.data[index]
        image1_path = osp.join(self.dataset_root, 'dataset-cfp', d[
            'img1_path'])
        image2_path = osp.join(self.dataset_root, 'dataset-cfp', d[
            'img2_path'])
        image1 = Image.open(image1_path).convert('RGB')
        image2 = Image.open(image2_path).convert('RGB')
        tag = d['pair_tag']
    
        if self.transforms is not None:
            # this converts from (HxWxC) to (CxHxW) as wel
            img1 = self.transforms(image1)
            img2 = self. transforms(image2)

        return img1, img2, tag

DEFINE DATA LOADER

In [0]:
from torch.utils import data

def get_dataloader(datapath, args, img_transforms=None, split="train"):

    if split == 'train':
        shuffle = True
        drop_last = True
    else:
        shuffle = False
        drop_last = False
    
    dataset = CFPDataset(datapath,
                         args,
                         split=split,
                         img_transforms=img_transforms,
                         dataset_root=osp.expanduser(args.dataset_root))
    
    data_loader = data.DataLoader(dataset,
                                  batch_size=args.batch_size,
                                  shuffle=shuffle,    
                                  num_workers=args.workers,
                                  pin_memory=True,
                                  drop_last=drop_last)
    return data_loader

DEFINE MODEL

In [0]:

import torch
from torch import nn
from torchvision.models import resnext50_32x4d

def l2norm(x):
  x = x / torch.sqrt(torch.sum(x**2, dim=-1, keepdim=True))
  return x

class SiameseCosine(nn.Module):
    """
    Siamese network
    """
    def __init__(self, pretrained=False):
        super(SiameseCosine, self).__init__()

        resnext50_32x4d_model = resnext50_32x4d(pretrained=pretrained)
        self.feat = resnext50_32x4d_model
        
    def forward(self, img1, img2):
        feat_1 = self.feat(img1)
        feat_1 = l2norm(feat_1)
        
        feat_2 = self.feat(img2)
        feat_2 = l2norm(feat_2)

      
        return feat_1, feat_2
      

DEFINE INFERENCE FUNCTIONS

In [0]:
import torch
from torchvision import transforms
from torch.nn import functional as nnfunc
from IPython.display import display
from PIL import Image
import numpy as np
from torchvision.transforms import ToPILImage


def similarity (vec1, vec2):
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-8)
    cos_a = cos (vec1, vec2)  
    return cos_a
  
def show_results(img1, img2, vec1, vec2, y, treshold):
 
    similarity_value = similarity(vec1, vec2)
   
    for image1, image2, value, label in zip (img1, img2, similarity_value, y):
        imtype = np.uint8
        mean = 0.5
        std = 0.5
        if value > treshold: 
            Predicted_label = 'Same Identity'
        else:
            Predicted_label = 'Different Identity'
        if label==1: 
            Label = 'Same Identity'
        else:
            Label = 'Different Identity'
        print("")
        print('Predicted label:', Predicted_label)
        print('Label:', Label)
        image1 = image1.cpu().float()
        image1 = (image1*std+mean)*255
        image1_numpy = image1.numpy()
        image1_numpy = np.transpose(image1_numpy, (1, 2, 0))
        display(Image.fromarray(image1_numpy.astype(imtype)))
        image2 = image2.cpu().float()
        image2 = (image2*std+mean)*255
        image2_numpy = image2.numpy()
        image2_numpy = np.transpose(image2_numpy, (1, 2, 0))
        display(Image.fromarray(image2_numpy.astype(imtype))) 
        
    return 

def inference(model, dataloader, epoch, device, tr):
    model.eval()
        
    for idx, (img1, img2, prob) in enumerate(dataloader):
        img1 =  img1.to('cuda:0')
        img2 = img2.to('cuda:0')
        prob = prob.float().to('cuda:0')  #label
        
        out1, out2 = model(img1, img2) #inputs images to model, executes model, returns features
        
        show_results(img1, img2, out1, out2, prob, tr)
        
        torch.cuda.empty_cache()
    return

STATE ARGUMENTS

In [0]:
args = Args()

DEFINE IMAGE TRANSFORMATIONS

In [0]:
train_transform=None
if args.data_aug == False:
  img_transforms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

else:
  img_transforms = transforms.Compose([transforms.Resize((224, 224)), 
                                        transforms.RandomHorizontalFlip(), 
                                        transforms.RandomRotation(20, resample=PIL.Image.BILINEAR), 
                                        transforms.ToTensor()])

    

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

LOAD DATA FOR INFERENCE

In [12]:
test_loader = get_dataloader(args.split_testdata, args,
                             img_transforms=val_transforms, split="test")

Dataset loaded
24 samples in the test dataset


STATE DEVICE

In [13]:
torch.cuda.is_available()

True

In [0]:
# check for CUDA
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

LOAD MODEL

In [15]:

model = SiameseCosine(pretrained=args.pretrained)

model = model.to(device) 


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 100441675/100441675 [00:03<00:00, 26453508.71it/s]


LOAD WEIGHTS

In [16]:
weights = osp.join(weights_dir,args.resume+'.pth')
epoch = 35 #extracted from experiment to reproduce
if args.resume:
    print(weights)
    checkpoint = torch.load(weights)
    model.load_state_dict(checkpoint['model'])
    # Set the start epoch if it has not been
    if not args.start_epoch:
        args.start_epoch = checkpoint['epoch']

gdrive/My Drive/2019_AIDL_TEAM4/colab_face_detection_siamesa/Presentacio/Demo/Weights_here/checkpoint_e63_40e_lr1e_3_SGD.pth


EXECUTE INFERENCE

In [17]:
# Inference

best_tr = 0.668 #extracted from experiment to reproduce
inference(model, test_loader, epoch, device=device, tr=best_tr)


Output hidden; open in https://colab.research.google.com to view.